In [1]:
import pandas as pd
from Bio import SeqIO 
import os,sys
import sgRNA_utils.sgRNA_primer_util as su
import module.sequencing_primer as sr
import module.from_gbFile_to_seq as fq
import module.parser_input_to_df as pf
import module.product_and_decorate_editingSeq as p_d_seq
import module.order as order
import warnings   
warnings.filterwarnings('ignore')  
import configparser
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
from sgRNA_utils.sgRNA_primer_config import config 
from loguru import logger
input_path = '/home/yanghe/program/edit_sequence_design/input/config.json'

In [2]:
#uha_dha_primer
def extract_uha_dha_primer(info_input_df, sgRNA):

    primer_template_for_u_d_ha_df = p_d_seq.create_primer_template(info_input_df,sgRNA)
    uha_dha_primer_df = p_d_seq.design_primer(primer_template_for_u_d_ha_df,'Name_Region','primer_template','u_d')
    uha_dha_primer_df = uha_dha_primer_df.rename(columns={'Region':'id'})
    uha_dha_primer_df = uha_dha_primer_df.join(uha_dha_primer_df.id.str.split(';',expand=True).rename(columns = {0:'Name',1:'Region',2:'Type'})).drop(columns='id')
    
    return uha_dha_primer_df

#uha_dha
def extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA):
    #整合进突变信息
    info_df = info_input_df[['name','region','seq_altered','type','ref','strand']].rename(columns={'name':'Name','region':'Region'})
    info_df.seq_altered.fillna('',inplace=True)
    uha_dha_info_primer_df = pd.merge(info_df,uha_dha_primer_df,on=['Name','Region'])

    #提取源生同源臂
    uha_dha_df = p_d_seq.create_uha_dha_df(uha_dha_primer_df) 
    #合并突变信息
    uha_dha_df = pd.merge(uha_dha_df,info_df,on=['Name','Region'])
    uha_dha_sgRNA_df = pd.merge(uha_dha_df,sgRNA,on=['Name','Region'],how='inner')

    return uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df,info_df
def two_plasmid_system_n20_enzyme_cut_seq(no_ccdb_uha_dha_sgRNA_df,promoter_seq,enzyme_df,enzyme_name):

    sgRNA_enzyme_df = enzyme_df[enzyme_df['name']==enzyme_name]
    cut_seq_len = sgRNA_enzyme_df.loc[0,'cut_seq_len']
  

    #N20片段引物退火获得
    temp_sgRNA_df = no_ccdb_uha_dha_sgRNA_df[['Name', 'Region','Target sequence', 'promoter_N20_terminator']]
    temp_sgRNA_df['Region'] = temp_sgRNA_df['Name'] +';'+ temp_sgRNA_df['Region']
    temp_sgRNA_df.drop(columns='Name',inplace=True)
    enzymeCutSeq_and_N20_df = p_d_seq.create_enzymeCutSeq_and_N20(temp_sgRNA_df, promoter_seq, enzyme_cut_len=cut_seq_len)
    #取出重要列
    enzymeCutSeq_and_N20_df = enzymeCutSeq_and_N20_df.rename(columns = {"Region":"ID"})
    enzymeCutSeq_and_N20_df = enzymeCutSeq_and_N20_df[[ "ID","Target sequence","enzymeCutSeq_and_N20"  ]]

    return enzymeCutSeq_and_N20_df

def region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json, first_primer_start_position):

    distance_dict = {}
    for k,v in sgRNA_region_json.items():
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        if first  > first_primer_start_position:
            distance = first - first_primer_start_position
        else:
            distance = sgRNA_plasmid_seq_len - first_primer_start_position + first
        distance_dict.update({distance:v})
    sorted_distance = sorted(distance_dict.keys())

    last_distance_dict = {}
    for i in range(len(sorted_distance)):
        v = distance_dict[sorted_distance[i]]
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        new_len = last - first

        if i != 0:
            v = distance_dict[sorted_distance[i-1]]
            arr = v.split(',')
            first = int(arr[0])
            last = int(arr[1])
            old_len = last - first
            min_distance = sorted_distance[i] - sorted_distance[i-1] -old_len
        else:
            min_distance = sorted_distance[i]  

        max_distance = min_distance + new_len

        last_distance_dict.update({f'primer{i+1}':(min_distance, max_distance)})

    print(last_distance_dict)

    return last_distance_dict

def read_chopchopInput_add_uha_dha(genome_path,chopchop_input,uha_dha_params):
    max_left_arm_seq_length = uha_dha_params['max_left_arm_seq_length']
    max_right_arm_seq_length = uha_dha_params['max_right_arm_seq_length']

    info_input_df = su.del_Unnamed(pd.read_csv(chopchop_input))

    def work(mun_id,geneid, mutation_pos_index):
        if mutation_pos_index - max_left_arm_seq_length < 0:
            error_message = "The length of upstream sequence of manipulation site of " + mun_id + " must be larger than sum of 'Max Length of UHA' and 'Max Length of UIS'."
            return error_message,error_message,error_message,error_message

        record = su.extract_seq_from_genome(genome_path,geneid)

        seq_uha_max_whole = str(record[
                        mutation_pos_index - max_left_arm_seq_length : mutation_pos_index
                        ])
        seq_dha_max_whole = str(record[
                                mutation_pos_index : mutation_pos_index + max_right_arm_seq_length
                                ])


        uha_upstream = str(  
                        record[
                            mutation_pos_index - max_left_arm_seq_length - 100 : mutation_pos_index - max_left_arm_seq_length
                        ]
                    )
        dha_downstream=str(
                        record[
                            mutation_pos_index + max_right_arm_seq_length  : mutation_pos_index + max_right_arm_seq_length  + 100
                        ]
                    )
        return  uha_upstream, dha_downstream, seq_uha_max_whole, seq_dha_max_whole

    info_df = su.lambda2cols(info_input_df,lambdaf=work,in_coln=['id','geneid','mutation_pos_index'],to_colns=['uha_upstream','dha_downstream','seq_uha_max_whole','seq_dha_max_whole'])

    return info_df



def sort_compose_primer(sgRNA_promoter_terminator_start,
                        primer_json,
                        primer_position_json,
                        gb_path,
                        sgRNA_plasmid_backbone,
                        n_20_label='N20',
                        ccdb_label='ccdB',
                        promoter_terminator_label = 'gRNA'):
    
    
    position = list(primer_position_json.values())
    first_list =  sorted([i for i in position if i > sgRNA_promoter_terminator_start])
    second_list =  sorted([i for i in position if i < sgRNA_promoter_terminator_start])
    first_list.extend(second_list)
    position_primer_json = {value: key for key, value in primer_position_json.items()}
    
    #引物数量
    primers_sum = len(first_list)*2 + 2
    primer_dict = {}
    i = 1

    first_last_primer_dict = first_left_last_right_primer_design(
                                        gb_path,
                                        ccdb_label,
                                        promoter_terminator_label,
                                        n_20_label,
                                        last_primer_num = int(primers_sum/2))

    primer_dict.update(first_last_primer_dict)
    #在质粒上取引物,同时设计引物
    for item in first_list:
        left_primer = primer_json[position_primer_json[item]]
        if item < 40:
            template = sgRNA_plasmid_backbone[-100:]+sgRNA_plasmid_backbone[:item]
        else:
            template = sgRNA_plasmid_backbone[:item]
        primer_result =  su.primer_design(seqId=item,seqTemplate=template,stype='right')
        if len(primer_result)<10:
            pass
        else:
            right_primer = primer_result['PRIMER_RIGHT_0_SEQUENCE']
            primer_dict.update({f"primer_r_seq_(5'-3')_{i}":right_primer})
            i = i + 1
            primer_dict.update({f"primer_f_seq_(5'-3')_{i}":left_primer})
    
    primer_dict_df = pd.DataFrame([primer_dict])
    
    return primer_dict_df,primers_sum

# 数据预处理

In [3]:
data = {

        "chopchop_input": "/home/yanghe/tmp/data_preprocessing/output/info_input.csv",   
        "sgRNA_result_path": "/home/yanghe/tmp/chopchop/output/sgRNA.csv",
        "edit_sequence_design_workdir":"/home/yanghe/tmp/edit_sequence_design/output/",
        "ref_genome":"/home/yanghe/program/data_preprocessing/input/GCA_000011325.1_ASM1132v1_genomic.fna",

        "one_plasmid_file_path": "/home/yanghe/program/edit_sequence_design/input/pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_ccdb_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_sgRNA_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",

        "uha_dha_config": {
            "max_right_arm_seq_length": 1050,
            "max_left_arm_seq_length": 1050,
            "min_left_arm_seq_length": 1000,   
            "min_right_arm_seq_length": 1000
        },

        
        "plasmid_label":{
            "ccdb_label":"ccdB",
            "promoter_terminator_label":"gRNA",
            "n_20_label":"N20"
        },
        
        "sgRNA_primer_json":{
            "primer3":"AACTATTTATCCAGTTGGTACAAAC"
        },
        "ccdb_primer_json":{
            "primer3":"AACTGATTCAGTCTGATTTCGCGGT"
        },
    
        "sgRNA_region_json":{
            "region1":"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt"
        },
        
        "ccdb_region_json":{
            "region1":"ATTGTGAGCGGATAACAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATACTCAATAGGCTTAGATATCGGCACAAATAGCGTCGGATGGGCGGTGATC"
        },
        
        "enzyme":{
            "enzyme_name":"BsaI",
            "gap_sequence":"A",
            "protection_sequence":"CCA"
        },
        
        "UHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "SEQ_ALTERED_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,  
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "DHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "UP_SGRNA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "DOWN_SGRNA_ARGS": {
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },

        "PLASMID_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,  
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "GENOME_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,  
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        }
       
    }

In [4]:
    chopchop_input = data['chopchop_input']

    #uha_dha参数
    uha_dha_params = data['uha_dha_config']
    
    # enzyme_path = parent_base_path +'/'+ data['enzyme_path']
    
    sgRNA_result_path = data['sgRNA_result_path']
    one_plasmid_file_path = data['one_plasmid_file_path']
    no_ccdb_plasmid = data['no_ccdb_plasmid']
    no_sgRNA_plasmid = data['no_sgRNA_plasmid']
    
    #plasmid label
    ccdb_label = data['plasmid_label']['ccdb_label']
    promoter_terminator_label = data['plasmid_label']['promoter_terminator_label']
    n_20_label = data['plasmid_label']['n_20_label']   
 
    #primer
    sgRNA_primer_json = data['sgRNA_primer_json']
    ccdb_primer_json = data['ccdb_primer_json']
    
    sgRNA_region_seq_json = data['sgRNA_region_json']
    ccdb_region_seq_json =  data['ccdb_region_json']
    
    #基因组
    genome_path = data['ref_genome']

    #配置引物参数
    # config.S_GLOBAL_ARGS = data['S_GLOBAL_ARGS']
    # config.Q_ARGS = data['Q_ARGS']
    config.UHA_ARGS = data['UHA_ARGS']
    config.SEQ_ALTERED_ARGS = data['SEQ_ALTERED_ARGS']
    config.DHA_ARGS = data['DHA_ARGS']
    config.UP_SGRNA_ARGS = data['UP_SGRNA_ARGS']
    config.DOWN_SGRNA_ARGS = data['DOWN_SGRNA_ARGS']

    config.PLASMID_Q_ARGS = data['PLASMID_Q_ARGS']
    config.GENOME_Q_ARGS = data['GENOME_Q_ARGS']   
    
    #配置输出参数
    output = data['edit_sequence_design_workdir']
    if not os.path.exists(output):
        os.makedirs(output)
     
    enzyme_path = './input/enzyme.csv'  
   

In [5]:
#1.read 编辑序列信息,给chopchop输入加uha、dha信息
info_input_df = read_chopchopInput_add_uha_dha(genome_path, chopchop_input, uha_dha_params)

#2.读取用户填的酶
enzyme_df = su.del_Unnamed(pd.read_csv(enzyme_path))
enzyme = data['enzyme']
enzyme_name = enzyme['enzyme_name']

#3.提取用户选择的sgRNA
sgRNA = p_d_seq.extract_sgRNA_from_chopchop(sgRNA_result_path)

#4.设计源生同源臂引物
uha_dha_primer_df = extract_uha_dha_primer(info_input_df, sgRNA)

#5.提取同源臂
uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df, info_df = extract_uha_dha(info_input_df, uha_dha_primer_df, sgRNA)


{}
{}
{}
{}
{}
{}
{}
{}
{}
{'PRIMER_LEFT_EXPLAIN': 'considered 8, ok 8', 'PRIMER_RIGHT_EXPLAIN': 'considered 408, low tm 132, not in any ok right region 172, ok 104', 'PRIMER_PAIR_EXPLAIN': 'considered 1, ok 1', 'PRIMER_LEFT_NUM_RETURNED': 1, 'PRIMER_RIGHT_NUM_RETURNED': 1, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 1, 'PRIMER_PAIR_0_PENALTY': 10.796287109339062, 'PRIMER_LEFT_0_PENALTY': 3.4730231907085454, 'PRIMER_RIGHT_0_PENALTY': 7.323263918630516, 'PRIMER_LEFT_0_SEQUENCE': 'GTTATTTCCGTGCGCGTTGGTGA', 'PRIMER_RIGHT_0_SEQUENCE': 'GCCTGCATTTATAAAGTGCTGACCC', 'PRIMER_LEFT_0': (0, 23), 'PRIMER_RIGHT_0': (1038, 25), 'PRIMER_LEFT_0_TM': 64.52697680929145, 'PRIMER_RIGHT_0_TM': 62.676736081369484, 'PRIMER_LEFT_0_GC_PERCENT': 52.17391304347826, 'PRIMER_RIGHT_0_GC_PERCENT': 48.0, 'PRIMER_LEFT_0_SELF_ANY_TH': 19.610968059260017, 'PRIMER_RIGHT_0_SELF_ANY_TH': 24.27068115227172, 'PRIMER_LEFT_0_SELF_END_TH': 5.11952728907761, 'PRIMER_RIGHT_0_SELF_END_TH': 15.625926837095221, 'P

In [11]:
def one_plasmid_system_design_by_user_region(uha_dha_sgRNA_df,sgRNA_region_seq_json,one_plasmid_file_path,sgRNA_plasmid_backbone,enzyme_df,enzyme_name):
    
    #序列转换成坐标
    region_json = su.convert_seq_cor(one_plasmid_file_path, sgRNA_region_seq_json,seq=sgRNA_plasmid_backbone)
    #坐标转转换成距离
    distance_dict = region_2_distance(len(sgRNA_plasmid_backbone), region_json, 0)

    #根据区域设计引物
    primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(0,sgRNA_plasmid_backbone,distance_dict)
    plasmid_primer_df = su.result_primer_list_to_df(primer_result_list)

    #加接头
    plasmid_primer_joint_df = p_d_seq.sgRNA_sgRNAprimer_merge(uha_dha_sgRNA_df,plasmid_primer_df,sgRNA_columns=['Name','Region'])
    primers_sum = len(plasmid_primer_df)
    plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df, enzyme_name, plasmid_primer_joint_df, primers_sum, primer_type='ccdb')
    plasmid_primer_joint_df = plasmid_primer_joint_df.drop(columns='Region').drop_duplicates()
    #修改df格式
    plasmid_primer_df = p_d_seq.plasmid_primer(plasmid_primer_joint_df)
    #添加索引
    plasmid_primer_df.insert(0,'index',plasmid_primer_df.index)
    plasmid_primer_df.reset_index(drop=True, inplace=True)

    plasmid_primer_df = p_d_seq.add_product_and_size(one_plasmid_file_path, plasmid_primer_df, enzyme_df, enzyme_name=enzyme_name,seq=sgRNA_plasmid_backbone)

    return  plasmid_primer_df


#one plasmid pcr primer  
def one_plasmid_system_pcr_design_primer(gb_path,
                                         info_df,
                                         uha_dha_sgRNA_df,
                                         uha_dha_info_primer_df,
                                         uha_dha_primer_df,
                                         enzyme_df,
                                         enzyme_name,
                                         plasmid_primer_desgin_type,
                                         seq_json):
    
    #创建新的质粒
    uha_dha_sgRNA_df, promoter_terminator_up_promoter_seq, promoter_terminator_down_terminator_seq, type_kind  = p_d_seq.create_new_plasmid(gb_path, uha_dha_sgRNA_df.copy(), ccdb_label='ccdB', promoter_terminator_label='gRNA', n_20_label='N20')

    #设计sgRNA、ccdb质粒引物
    n20up_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_up_template','Target sequence','Rev Target sequence']]
    n20up_primer_template['Region'] = n20up_primer_template['Name'] +';'+ n20up_primer_template['Region']
    n20up_primer_df = p_d_seq.design_primer(n20up_primer_template,'Region','n20_up_template','sgRNA')
    n20up_primer_df = pd.merge(n20up_primer_template[['Region','Target sequence','Rev Target sequence']],n20up_primer_df,on=['Region'],how='inner')

    #ccdb、sgrna质粒引物加接头
    n20up_primer_df = p_d_seq.add_joint_sgRNA_primer(n20up_primer_df,enzyme_df,enzyme_name,'',stype='n20up_primer_joint')


     #质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物  
    if plasmid_primer_desgin_type == 2: 
        n20down_primer_template = uha_dha_sgRNA_df[['Name','Region','n20_down_template','Target sequence','Rev Target sequence']]
        n20down_primer_template['Region'] = n20down_primer_template['Name'] +';'+ n20down_primer_template['Region']
        n20down_primer_df = p_d_seq.design_primer(n20down_primer_template,'Region','n20_down_template','sgRNA')
        n20down_primer_df = pd.merge(n20down_primer_template[['Region','Target sequence','Rev Target sequence']],n20down_primer_df,on=['Region'],how='inner')
        #加接头
        n20down_primer_df = p_d_seq.add_joint_sgRNA_primer(n20down_primer_df,enzyme_df,enzyme_name,'',stype='n20down_primer_joint')

    elif plasmid_primer_desgin_type == 1:
        plasmid_backbone = promoter_terminator_down_terminator_seq
        n20down_primer_df =  one_plasmid_system_design_by_user_region(uha_dha_sgRNA_df,seq_json,gb_path,plasmid_backbone,enzyme_df,enzyme_name)
    elif plasmid_primer_desgin_type == 3:
        plasmid_backbone = promoter_terminator_down_terminator_seq
        primer_position_json, sgRNA_failture_primer = p_d_seq.check_locate_primer(plasmid_backbone, seq_json)
        n20down_primer_df = one_plasmid_system_design_by_user_primer(seq_json,primer_position_json,gb_path,plasmid_backbone,enzyme_df,enzyme_name)


    seq_altered_primer_template = info_df[info_df.seq_altered.apply(lambda x:len(x)>120)][['Name','Region','seq_altered']]
    seq_altered_primer_template['Region'] = seq_altered_primer_template['Name'] + seq_altered_primer_template['Region']
    seq_altered_primer_df = p_d_seq.design_primer(seq_altered_primer_template,'Region','seq_altered',stype='seq_altered')

    #给同源臂引物加接头:uha取promoter_terminator_down_terminator_seq尾部反义4bp，dha取头promoter_terminator_up_promoter_seq正义4bp
    uha_dha_primer_df = p_d_seq.add_joint_sgRNA_primer(uha_dha_info_primer_df,enzyme_df,enzyme_name,promoter_terminator_down_terminator_seq, promoter_terminator_up_promoter_seq, stype = 'u_d_primer_joint')

    #seq_altered_primer加接头
    seq_altered_primer_df = p_d_seq.add_joint_sgRNA_primer(seq_altered_primer_df,enzyme_df,enzyme_name,'',stype='seq_altered_primer_joint')

    #分别提取，修饰后的uha、dha引物
    in_col = ['Name','Region',"primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']
    ou_col = ['Name','Region',"u_primer_f_seq_(5'-3')","u_primer_r_seq_(5'-3')",'UHA','UHA_size',"d_primer_f_seq_(5'-3')","d_primer_r_seq_(5'-3')",'DHA','DHA_size']
    uha_primer_df, dha_primer_df = p_d_seq.create_uha_dha_primer_df(uha_dha_primer_df, in_col, ou_col)

    n20down_primer_p_df = n20down_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]
    n20up_primer_p_df = n20up_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]
    seq_altered_primer_df = seq_altered_primer_df[["Region","primer_f_seq_(5'-3')_joint","primer_r_seq_(5'-3')_joint",'product_value_joint','product_size_joint']]

    return uha_dha_sgRNA_df,uha_primer_df, dha_primer_df, n20down_primer_p_df, n20up_primer_p_df, seq_altered_primer_df,type_kind


In [12]:
uha_dha_sgRNA_df,uha_primer_df,dha_primer_df,n20down_primer_p_df,n20up_primer_p_df, seq_altered_p_df,type_kind = one_plasmid_system_pcr_design_primer(                                                                                                                            
                                                                                                                                        one_plasmid_file_path,
                                                                                                                                        info_df,
                                                                                                                                        uha_dha_sgRNA_df,
                                                                                                                                        uha_dha_info_primer_df,
                                                                                                                                        uha_dha_primer_df,
                                                                                                                                        enzyme_df,
                                                                                                                                        enzyme_name,
                                                                                                                                        1,
                                                                                                                                        sgRNA_region_seq_json
                                                                                                                                        )
    

-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
8949 9413
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
9002 9022
tag: 0 marker 5057 5363
5057 5363 8949 9413
kjfdhsgjkhd
启动子-n20-终止子上游序列： TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCAATGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGA

seq_altered_left_153019_ecoil_ybeL_insBA000036.3:153019-153020: CCA GGTCTC A CCAGGTCTCAATGAACAAGGTTGCTCAATATTACC
seq_altered_right_153019_ecoil_ybeL_insBA000036.3:153019-153020: GGTCCAGAGT GGTCCAGAGTTTACGGCTCAAACGGGCGTCT ; CCA GGTCTC A
seq_altered_left_Cgl0851_ecoli_pgi_subBA000036.3:907755-909378: CCA GGTCTC A CCAGGTCTCAatgaaaaacatcaatccaacgcaga
seq_altered_right_Cgl0851_ecoli_pgi_subBA000036.3:907755-909378: GGTCCAGAGT GGTCCAGAGTttaaccgcgccacgctttatagc ; CCA GGTCTC A
Index(['Name', 'Region', 'primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')',
       'primer_f_Tm', 'primer_r_Tm', 'primer_f_seq_(5'-3')_joint',
       'primer_r_seq_(5'-3')_joint', 'product_value', 'product_size',
       'product_value_joint', 'product_size_joint', 'Type'],
      dtype='object')


In [30]:
def one_plasmid_system_sequencing_design_primer(type_kind,uha_dha_sgRNA_df):
    print(type_kind,"1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp","2:代表启动子上游的序列小于600bp","3:代表终止子上游序列小于600bp")

    if type_kind == 3:
        #载体测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA']+sequencing_primer_df['seq_altered']+sequencing_primer_df['DHA']+sequencing_primer_df['promoter_N20_terminator_up'] + sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')

    elif type_kind == 1:
        #载体测序
        #uha_dha测序
        sequencing_primer_df=uha_dha_sgRNA_df[['Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                    'plasmid', 'promoter_N20_terminator',
                                    'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA']
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template1 = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df1 = p_d_seq.create_sequencing_primer(sequencing_primer_template1,sr,'plasmid','plasmid_sequencing_region')
        #promoter_N20_terminator测序
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['promoter_N20_terminator']
        sequencing_primer_template2 = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df2 = p_d_seq.create_sequencing_primer(sequencing_primer_template2,sr,'plasmid','plasmid_sequencing_region')
        plasmid_sequencing_primer_df = p_d_seq.merge_sequencing_result(plasmid_sequencing_primer_df1, plasmid_sequencing_primer_df2)
        
        sequencing_primer_template = pd.merge(sequencing_primer_template1,sequencing_primer_template2,on=['Name','Region'],how='inner')
        
        
    elif type_kind == 2:
        #载体测序
        #uha_dha测序
        sequencing_primer_df=uha_dha_sgRNA_df[[ 'Name', 'Region', 'UHA', 'UHA_size', 'DHA', 'DHA_size',
                                                'plasmid', 'promoter_N20_terminator',
                                                'promoter_N20_terminator_up', 'promoter_N20_terminator_down','seq_altered']]
        sequencing_primer_df['plasmid_sequencing_region'] = sequencing_primer_df['UHA'] + sequencing_primer_df['seq_altered'] + sequencing_primer_df['DHA'] + sequencing_primer_df['promoter_N20_terminator_up'] + sequencing_primer_df['promoter_N20_terminator'] 
        sequencing_primer_df['Region'] = sequencing_primer_df['Name']+';'+sequencing_primer_df['Region']
        sequencing_primer_template = sequencing_primer_df[['Name','Region','plasmid_sequencing_region','plasmid']]
        plasmid_sequencing_primer_df = p_d_seq.create_sequencing_primer(sequencing_primer_template,sr,'plasmid','plasmid_sequencing_region')

    return plasmid_sequencing_primer_df, sequencing_primer_template

In [29]:
type_kind

1

In [26]:
plasmid_sequencing_primer_df,sequencing_primer_template = one_plasmid_system_sequencing_design_primer(type_kind,uha_dha_sgRNA_df)

1 1:代表启动子上游的序列大于600bp,同时终止子下游大于600bp 2:代表启动子上游的序列小于600bp 3:代表终止子上游序列小于600bp
0 2554
2954
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'GAGCGCAACTCGGATGATCC', 'SEQUENCING_PRIMER_3_TM': 61.21822976233301, 'SEQUENCING_PRIMER_4': 'CGAACGCCTGCGCAATGGC', 'SEQUENCING_PRIMER_4_TM': 65.74288342382255, 'SEQUENCING_PRIMER_5': 'CGGCTCACATCAAGTCGTGCC', 'SEQUENCING_PRIMER_5_TM': 64.1446146725608}, {})
0 2066
2466
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACGCTG', 'SEQUENCING_PRIMER_1_TM': 61.16590711741475, 'SEQUENCING_PRIMER_2': 'GAGCTGATACCGCTCGCCGC', 'SEQUENCING_PRIMER_2_TM': 65.96384986687252, 'SEQUENCING_PRIMER_3': 'TCGACGTCTGCAGGAAATGGCA', 'SEQUENCING_PRIMER_3_TM': 64.74457393037193, 'SEQUENCING_PRIMER_4': 'TGCGCTACCACAAGATCGTGCT', 'SEQUENCING_PRIMER_4_TM': 64.78597081263268}, {})
0 2059
2459
({'SEQUENCING_PRIMER_1': 'TTGCTCACCCAGAAACG

In [16]:
sequencing_primer_template

,Name,Region,plasmid_sequencing_region,plasmid
0,153019_ecoil_ybeL_ins,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAG...
1,Cgl0006_1176_G_A_sub,Cgl0006_1176_G_A_sub;BA000036.3:6529-6530,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,AAGCGCCCGGGCATGTACATCGGTTCAACTGGACCGCGTGGACTGC...
2,Cgl0141_cds_del,Cgl0141_cds_del;BA000036.3:153222-155613,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGC...
3,Cgl0591_-1_Ppgk_promoter_ins,Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,GGAATCACAGGCAGGGTGGGAGATCTTCCACGCCTTGCAGGTGAAA...
4,Cgl0851_ecoli_pgi_sub,Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,AACCCAAACCCTGCCCGAAGGAACTCAACGCAGAAGCACCAGCACT...
5,Cgl1386_327_18to15_sub,Cgl1386_327_18to15_sub;BA000036.3:1454441-1454459,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,GGCCCAATGTGGTCCCTCGGCGGCAACTACTCTGAAGGTTGGGAGT...
6,Cgl1436_1113_CAA_del,Cgl1436_1113_CAA_del;BA000036.3:1514089-1514092,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,CAACAGCTCCTCCTGGGCCGCCTTAATTGCCCCGACACGCGCATGA...
7,Cgl1790_1647_TCC_sub,Cgl1790_1647_TCC_sub;BA000036.3:1899017-1899020,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,GCAGGACAATAGCGTTACCCTGGTCAGCGACACGACGAGTGAGCTC...
8,Cgl2342_213_GCA_ins,Cgl2342_213_GCA_ins;BA000036.3:2484874-2484875,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TTACCGGACACGCGCAACACATCTTCAGGTTCACAACCCACAATGT...


In [28]:
sequencing_primer_df

,Name,Region,UHA,UHA_size,DHA,DHA_size,plasmid,promoter_N20_terminator,promoter_N20_terminator_up,promoter_N20_terminator_down,seq_altered,plasmid_sequencing_region
0,153019_ecoil_ybeL_ins,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAG...,1034,GCTTTTAAGTTTTCTCGGTGATCCGTAACGAGAAGTTGAACAGCAA...,1037,AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAG...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,ATGAACAAGGTTGCTCAATATTACCGTGAACTGGTTGCGTCACTGA...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
1,Cgl0006_1176_G_A_sub,Cgl0006_1176_G_A_sub;BA000036.3:6529-6530,AAGCGCCCGGGCATGTACATCGGTTCAACTGGACCGCGTGGACTGC...,1026,GTTATTTCCGTGCGCGTTGGTGACCCACAGTTCGAAGGCCAGACCA...,1039,AAGCGCCCGGGCATGTACATCGGTTCAACTGGACCGCGTGGACTGC...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,A,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
2,Cgl0141_cds_del,Cgl0141_cds_del;BA000036.3:153222-155613,TCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGC...,1035,CTACATGCGATTCTTGGTGCGCGCAGTGGCAGGAGCGGTCCATGGA...,1024,TCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGC...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
3,Cgl0591_-1_Ppgk_promoter_ins,Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469...,GGAATCACAGGCAGGGTGGGAGATCTTCCACGCCTTGCAGGTGAAA...,1050,ATGACTAAAACACTTGGTTCCCTTCAGCTGGAAGAAATCACGCTGA...,1020,GGAATCACAGGCAGGGTGGGAGATCTTCCACGCCTTGCAGGTGAAA...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,TTAATCAATACAATTGAATACCGGTGCCAGCGCCACACAATGTGTA...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
4,Cgl0851_ecoli_pgi_sub,Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378,AACCCAAACCCTGCCCGAAGGAACTCAACGCAGAAGCACCAGCACT...,1036,CTACCTATTTGCGCGGTACCACTTAATCAGTGAATCAGTGGAAGAA...,1035,AACCCAAACCCTGCCCGAAGGAACTCAACGCAGAAGCACCAGCACT...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,atgaaaaacatcaatccaacgcagaccgctgcctggcaggcactac...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
5,Cgl1386_327_18to15_sub,Cgl1386_327_18to15_sub;BA000036.3:1454441-1454459,GGCCCAATGTGGTCCCTCGGCGGCAACTACTCTGAAGGTTGGGAGT...,1027,GCTGAGATCGAAAAGCGTGTTGCTGAAGCCTCACGCATTCTGCAGC...,1032,GGCCCAATGTGGTCCCTCGGCGGCAACTACTCTGAAGGTTGGGAGT...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,GCGTGGCGCGGTTAA,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
6,Cgl1436_1113_CAA_del,Cgl1436_1113_CAA_del;BA000036.3:1514089-1514092,CAACAGCTCCTCCTGGGCCGCCTTAATTGCCCCGACACGCGCATGA...,1023,CAAAGTGGTGGTGATGCGCTTCGGGATTCCCGCAGGTTCCGCCACT...,1029,CAACAGCTCCTCCTGGGCCGCCTTAATTGCCCCGACACGCGCATGA...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
7,Cgl1790_1647_TCC_sub,Cgl1790_1647_TCC_sub;BA000036.3:1899017-1899020,GCAGGACAATAGCGTTACCCTGGTCAGCGACACGACGAGTGAGCTC...,1024,GACTTCTTCTTCGCGCCCAATAATCTCACGCACCGGTACAGCAACA...,1025,GCAGGACAATAGCGTTACCCTGGTCAGCGACACGACGAGTGAGCTC...,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...,TGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTT...,AAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAA...,TTA,TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTG...
8,Cgl2342_213_GCA_ins,Cgl2342_213_GCA_ins;BA000036.3:2484874-2484875,TTACCGGACACGCGCAACACATCTTCAGGTTCACAACCCACAATGT...,1020,CCTGAAATTTCCGTCATGCCGACCGAATCAATGGCACGGCTGATCT...,1023,TTACCGGACACGCGCAACACATCTTCAGGTTCACAACC

In [20]:
sequencing_primer_template.loc[0,'plasmid_sequencing_region']

'TTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTGTAATTCACGTTCGCTTTTAAGTTTTCTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCTTTTTTTCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCCTGAGTAGGACAAATCCGCCGGGAGCGGATTTGAACGTTGCGAAGCAACGGCCCGGAGGGTGGCGGGCAGGACGCCCGCCATAAACTGCCAGGCATCAAATTAAGCAGAAGGCCATCCTGACGGATGGCCTTTTTGCGTTTCTACAAACTCTTTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAATTAATTCCGCTAGATGACGTGCGGCTTCGA'

In [33]:
sequencing_primer_template.loc[0,'plasmid']


'AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAGTCGGACTCATCGCAGCCTACGACGGCATCTTCGGACCCGGAACCGGCATGTTCCTCATCATGGCGTTCACCGCACTGCTCTCCCAAAATTTCCTGTCCTCCGCAGCCATGGCGAAGGTCGTAAACACCGCAACAAACCTGGGTGCGCTAATTGTATTCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGCAGTCGCCAATGTCGCAGGCGCACAACTCGGTGCCCGAACGGTGCTTGGTGGCGGTACCAGGCTAATTAGATACGCACTACTAACCCTGGTTGTCGTCATGAGCGTCTACCTCACCTGGCAACAAATCCAAGGAATGTAGATAAGTCGGGGCAAACTCTAAGAAATCATCCTGGTTTGTGGCAGGGAACATTCCTAGGATGGGAGGCATGGCAAAGACAAAACTAGAAAGCATCTCCGAGCGCAACTCGGATGATCCGCTTAGTTACCCCGACCTGACTCTTGCACCAATATCCACAACAGTCACAGTCGTGGTCGCCGAAGACACCCCAAACGATAACTTCTCAGAACTTCTCCTCGGCGCAGCAGAATCAACCCGTCTTTTAGGACTCGACGAGCTCCACATCATCGCTCCATCAGTCCACCTACCCGCTCTCGCAGTAGCAGCCGCCGACATCGCACACCATCTGCCAGAGAAATTCCAATTCTGTGAAGCCGAAACCTGCACTCATCTCCACCCCGACGATGACACCTATCTCACGGCTGAATCCGTTGCTCAATTGGGTACGAAGCTGAAGTCTGCGTAGTTGTTAGGAGCCACCACAGACCCGTAGATTCTTGGGCGGTGTGGTGAATCCTTGCCCTGGAACCTTGCGGGAGACTCTCAGAAATGAGTCATTGGGGCTTGGGGTGTGGTGAAACGCAAACATCCGTCCCCGGACCGCGGTAAAAGGCCGATTTTGATCGAAAATTC

In [32]:
uha_dha_df.loc[0,'UHA']

'AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAGTCGGACTCATCGCAGCCTACGACGGCATCTTCGGACCCGGAACCGGCATGTTCCTCATCATGGCGTTCACCGCACTGCTCTCCCAAAATTTCCTGTCCTCCGCAGCCATGGCGAAGGTCGTAAACACCGCAACAAACCTGGGTGCGCTAATTGTATTCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGCAGTCGCCAATGTCGCAGGCGCACAACTCGGTGCCCGAACGGTGCTTGGTGGCGGTACCAGGCTAATTAGATACGCACTACTAACCCTGGTTGTCGTCATGAGCGTCTACCTCACCTGGCAACAAATCCAAGGAATGTAGATAAGTCGGGGCAAACTCTAAGAAATCATCCTGGTTTGTGGCAGGGAACATTCCTAGGATGGGAGGCATGGCAAAGACAAAACTAGAAAGCATCTCCGAGCGCAACTCGGATGATCCGCTTAGTTACCCCGACCTGACTCTTGCACCAATATCCACAACAGTCACAGTCGTGGTCGCCGAAGACACCCCAAACGATAACTTCTCAGAACTTCTCCTCGGCGCAGCAGAATCAACCCGTCTTTTAGGACTCGACGAGCTCCACATCATCGCTCCATCAGTCCACCTACCCGCTCTCGCAGTAGCAGCCGCCGACATCGCACACCATCTGCCAGAGAAATTCCAATTCTGTGAAGCCGAAACCTGCACTCATCTCCACCCCGACGATGACACCTATCTCACGGCTGAATCCGTTGCTCAATTGGGTACGAAGCTGAAGTCTGCGTAGTTGTTAGGAGCCACCACAGACCCGTAGATTCTTGGGCGGTGTGGTGAATCCTTGCCCTGGAACCTTGCGGGAGACTCTCAGAAATGAGTCATTGGGGCTTGGGGTGTGGTGAAACGCAAACATCCGTCCCCGGACCGCGGTAAAAGGCCGATTTTGATCGAAAATTC

In [31]:
uha_dha_df.loc[0,'DHA']

'GCTTTTAAGTTTTCTCGGTGATCCGTAACGAGAAGTTGAACAGCAAACCCGCAGCAACTGTTCAGAAATCACAACAGTCCCAATGACCTCAAAATCGGCCCCTTCTAGAATCGCCTCTCAGGGCCTTTGAGCGCTAAATTGGGGCAACAGGTCACTGCGTACAAAGAGGGCCTTAAATCGGCTCACATCAAGTCGTGCCAAAACTACATGCGATTCTTGGTGCGCGCAGTGGCAGGAGCGGTCCATGGATCCTCCGGCCACGGGTGTTTGGGGTACCGGCCCCGCATTTCTGCACGGACTTGGCTGTACGGGCCAGACCAGAAACTACGCAGGTCATCGGTAACTGCAAGAGGTCTTCCCGCAGGTGACAGGAGATGAAATTGCACCGGGATTCCGCAGAGCTGCGGGGATTCTTCTAATCCGAAGCATTCTTGTAATTTCACTCGTATAACTGGCCTGCCTGATGAATAATCTAGTCGGTGCTGATTCCCGCTGGGTACGGATAGGTGGGAAGGAGCGAATTCTTCAAAGTTGGTTGCTTCAGGCCAGGGGAGAAGTCGCTGGAGTGCGGGATACATATCGATATTGTTTAGTCGTGTTCCGTGGCTGAGCGCCTCAATTTCTGGAGATAACCACAGGTGCGGGTCCGCAGTTTCAATATCTGGCCAGGGTTCGCCACGGTGTTCGTGGATGAATTTCAGTCGGTCACGCAAGGATGCCGCTTTGTCGGAGAAGTGGAATAGGTCGATTCCGCCTTTGGCAAGGGCTGTGGCGATTGTTTCCGAGGCTTCGGCGGGTGTTGGTTTTGTCGGAGTGGAACTTAATTCGATCGCGCCTGCTGCTTTGACCTTCCGGGCTTGGACTTTTCCGTTGACAAAAATCGCGCGGGTTTCTTCCACGACTCCGATGATGTCTATGGCATCGTCCTCTGAAATGCGGGCGGCGGCGCGGATGATGGCAGAATTTTGTGAGCGGTTAATTGCAGCAACGGAGATCCAT

In [5]:
# with open(input_path, "r") as f:
#         data = json.load(f)






chopchop_input = data['chopchop_input']
    # enzyme_path = parent_base_path +'/'+ data['enzyme_path']
    
    
uha_dha_params = data['uha_dha_config']

sgRNA_result_path = data['sgRNA_result_path']
one_plasmid_file_path = data['one_plasmid_file_path']
no_ccdb_plasmid = data['no_ccdb_plasmid']
no_sgRNA_plasmid = data['no_sgRNA_plasmid']

 #plasmid label
ccdb_label = data['plasmid_label']['ccdb_label']
promoter_terminator_label = data['plasmid_label']['promoter_terminator_label']
n_20_label = data['plasmid_label']['n_20_label']

#primer
sgRNA_primer_json = data['sgRNA_primer_json']
ccdb_primer_json = data['ccdb_primer_json']

sgRNA_region_seq_json = data['sgRNA_region_json']
ccdb_region_seq_json = data['ccdb_region_json']

#配置引物参数
config.S_GLOBAL_ARGS = data['S_GLOBAL_ARGS']
config.Q_ARGS = data['Q_ARGS']

#genome
genome_path = data['ref_genome']

   #配置输出参数
output = data['edit_sequence_design_workdir']
if not os.path.exists(output):
    os.makedirs(output)


enzyme_path = data['enzyme_path']
enzyme_name = data['enzyme_name']
enzyme_df = su.del_Unnamed(pd.read_csv(enzyme_path))

In [6]:
sgRNA_region_seq_json
ccdb_region_seq_json
sgRNA_primer_json
ccdb_primer_json

{'region1': 'tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt'}

In [ ]:
def check_primer(gb_path,seq_json):
        

In [10]:
ccdb_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'

In [11]:
with open(ccdb_gb_path, 'r')as f:
    gb_text = f.read()
    

'LOCUS       Exported               11188 bp ds-DNA     circular SYN 29-DEC-2022\nDEFINITION  Shuttle vector pXMJ19 (Corynebacterium glutamicum / E. coli).\nACCESSION   AJ133195\nVERSION     .\nKEYWORDS    .\nSOURCE      synthetic DNA construct\n  ORGANISM  synthetic DNA construct\nREFERENCE   1  (bases 1 to 11188)\n  AUTHORS   Jakoby MJ, Ngouto-Nkili CE, Burkovski A.\n  TITLE     Construction and application of new Corynebacterium glutamicum \n            vectors\n  JOURNAL   BioTechniques 13, 437-441 (1999)\nREFERENCE   2  (bases 1 to 11188)\n  AUTHORS   Jakoby MJ.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (22-FEB-1999) Jakoby M.J., Institut fuer Biochemie, \n            Universitaet zu Koeln, Zuelpicher Str. 47, 50674 Koeln, GERMANY\nREFERENCE   3  (bases 1 to 11188)\n  AUTHORS   .\n  TITLE     Direct Submission\n  JOURNAL   Exported Dec 29, 2022 from SnapGene 4.2.4\n            http://www.snapgene.com\nCOMMENT     This file is created by Vector NTI\n            http://w

In [ ]:
# 1.read 编辑序列信息
info_input_df = read_chopchopInput_add_uha_dha(genome_path,chopchop_input, uha_dha_params)

In [87]:
# 3.提取用户选择的sgRNA
sgRNA = p_d_seq.extract_sgRNA_from_chopchop(sgRNA_result_path)

In [88]:
# 4.设计源生同源臂引物
uha_dha_primer_df = extract_uha_dha_primer(info_input_df, sgRNA)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{'PRIMER_LEFT_EXPLAIN': 'considered 8, ok 8', 'PRIMER_RIGHT_EXPLAIN': 'considered 408, low tm 132, not in any ok right region 172, ok 104', 'PRIMER_PAIR_EXPLAIN': 'considered 1, ok 1', 'PRIMER_LEFT_NUM_RETURNED': 1, 'PRIMER_RIGHT_NUM_RETURNED': 1, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 1, 'PRIMER_PAIR_0_PENALTY': 10.796287109339062, 'PRIMER_LEFT_0_PENALTY': 3.4730231907085454, 'PRIMER_RIGHT_0_PENALTY': 7.323263918630516, 'PRIMER_LEFT_0_SEQUENCE': 'GTTATTTCCGTGCGCGTTGGTGA', 'PRIMER_RIGHT_0_SEQUENCE': 'GCCTGCATTTATAAAGTGCTGACCC', 'PRIMER_LEFT_0': (0, 23), 'PRIMER_RIGHT_0': (1038, 25), 'PRIMER_LEFT_0_TM': 64.52697680929145, 'PRIMER_RIGHT_0_TM': 62.676736081369484, 'PRIMER_LEFT_0_GC_PERCENT': 52.17391304347826, 'PRIMER_RIGHT_0_GC_PERCENT': 48.0, 'PRIMER_LEFT_0_SELF_ANY_TH': 19.610968059260017, 'PRIMER_RIGHT_0_SELF_ANY_TH': 24.27068115227172, 'PRIMER_LEFT_0_SELF_END_TH': 5.11952728907761, 'PRIMER_RIGHT_0_SELF_END_TH': 15.625926837095221, 'P

In [89]:
# 5.提取同源臂
uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df, info_df = extract_uha_dha(info_input_df,uha_dha_primer_df,sgRNA)

Index(['primer_f_seq_(5'-3')', 'primer_r_seq_(5'-3')', 'primer_f_Tm',
       'primer_r_Tm', 'product_size', 'product_value', 'Name', 'Region',
       'Type'],
      dtype='object')


# 双质粒设计

## 构建两个新质粒

In [90]:
no_ccdb_uha_dha_sgRNA_df,\
sgRNA_plasmid_backbone,\
promoter_seq,\
terminator_seq,\
sgRNA_promoter_terminator= p_d_seq.create_new_plasmid(no_ccdb_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)


no_sgRNA_uha_dha_ccdb_df,\
ccdB_plasmid_backbone,\
ccdB_promoter_terminator_up_seq  = p_d_seq.create_new_plasmid(no_sgRNA_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [8]:
#酶切退火方式获得质粒和n20
enzymeCutSeq_and_N20_df = two_plasmid_system_n20_enzyme_cut_seq(no_ccdb_uha_dha_sgRNA_df,promoter_seq,enzyme_df,enzyme_name)

In [9]:
#选择引物设计类型
#质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物

In [91]:
plasmid_primer_desgin_type = 1

## 给定质粒的区域设计引物

In [93]:
sgRNA_region_json = {'region1': '371,570', 'region2': '11307,200'}

In [95]:
#将sgRNA质粒分割成：promoter_terminator_up_seq、promoter_seq、n20_coordinate_seq、terminator_seq、promoter_terminator_down_seq
sgRNA_plasmid_seq, sgRNA_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_ccdb_plasmid,
                                                                                        ccdb_label,
                                                                                        promoter_terminator_label,
                                                                                        n_20_label)
#将ccdb质粒分割成：ccdb_up_seq、ccdb、ccdb_down_seq
ccdb_plasmid_seq, ccdb_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_sgRNA_plasmid,
                                                                                      ccdb_label,
                                                                                      promoter_terminator_label,
                                                                                      n_20_label)


-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [120]:
#针对sgRNA质粒的操作：
#确定第一条引物的位置在质粒上的位置：先找到启动子终止子的绝对位置，再确定相对位置，再确定绝对位置
promoter_terminator_seq = sgRNA_plasmid_region_seq['promoter_seq']+sgRNA_plasmid_region_seq['n20_coordinate_seq']+sgRNA_plasmid_region_seq['terminator_seq']
promoter_terminator_start = sgRNA_plasmid_seq.find(promoter_terminator_seq)
promoter_terminator_end = promoter_terminator_start + len(promoter_terminator_seq)
first_primer_position_in_promoter_terminator = promoter_terminator_seq.find(sgRNA_plasmid_region_seq['terminator_seq'])
first_primer_start_position = promoter_terminator_start + first_primer_position_in_promoter_terminator
#将用户提供的质粒设计区域转换成，各个区域之间的最小，最大距离
sgRNA_plasmid_seq_len = len(sgRNA_plasmid_seq)
sgRNA_distance_dict = region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json,first_primer_start_position)


{'primer1': (2591, 2830), 'primer2': (171, 370)}


In [121]:
#针对区域设计引物，特殊处理第一条和最后一条引物
sgRNA_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position, sgRNA_plasmid_seq, sgRNA_distance_dict, 20)
sgRNA_plasmid_primer_df =  su.result_primer_list_to_df(sgRNA_plasmid_primer_result_list)

570 8716 hdsfjkgfhjgjghfj


In [122]:
sgRNA_plasmid_primer_df

,Region,primer_f_seq_(5'-3'),primer_r_seq_(5'-3'),primer_f_Tm,primer_r_Tm,product_size,product_value
0,1,GTTTTAGAGCTAGAAATAGCAAGT,CGCTGTCTCTCCACTGTCAA,54.971016,59.684749,2830,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...
0,2,GAAGCGACTCGTCTCAAACGGACA,TGTCCACATCACTATTATCAGGA,65.080717,56.526896,370,GAAGCGACTCGTCTCAAACGGACAGCTCGTAGAAGGTATACACGTC...
0,3,AACTATTTATCCAGTTGGTACAAAC,TGAATTACACTGTACCTGTTGCGT,55.566353,60.741235,8126,GAAGCGACTCGTCTCAAACGGACAGCTCGTAGAAGGTATACACGTC...


In [100]:
sgRNA_plasmid_seq_len

11346

In [101]:
first_primer_start_position

8716

In [102]:
    distance_dict = {}
    for k,v in sgRNA_region_json.items():
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        if first  > first_primer_start_position:
            distance = first - first_primer_start_position
        else:
            distance = sgRNA_plasmid_seq_len - first_primer_start_position + first
        distance_dict.update({distance:v})
    sorted_distance = sorted(distance_dict.keys())

In [103]:
sorted_distance


[2591, 3001]

In [104]:
distance_dict

{3001: '371,570', 2591: '11307,200'}

In [10]:



#针对ccdb载体质粒的操作：
ccdb_start = ccdb_plasmid_seq.find(ccdb_plasmid_region_seq['ccdb'])
ccdb_end = ccdb_start + len(ccdb_plasmid_region_seq['ccdb'])
first_primer_start_position = ccdb_end
ccdb_plasmid_seq_len = len(ccdb_plasmid_seq)
ccdb_distance_dict = region_2_distance(ccdb_plasmid_seq_len, ccdb_region_json,first_primer_start_position)
ccdb_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position,ccdb_plasmid_seq, ccdb_distance_dict,len(ccdb_plasmid_region_seq['ccdb']))
ccdb_plasmid_primer_df = su.result_primer_list_to_df(ccdb_plasmid_primer_result_list)

#sgRNA质粒引物加接头
sgRNA_plasmid_primer_joint_df = p_d_seq.sgRNA_sgRNAprimer_merge(uha_dha_sgRNA_df, sgRNA_plasmid_primer_df)
sgRNA_primers_sum=len(sgRNA_plasmid_primer_df)
sgRNA_plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df,
                                                                 enzyme_name,
                                                                 sgRNA_plasmid_primer_joint_df,
                                                                 sgRNA_primers_sum,
                                                                 primer_type='sgRNA')

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


## 给定质粒上的引物

In [16]:
#根据给定sgRNA质粒设计
uha_dha_sgRNA_df, sgRNA_plasmid_backbone, sgRNA_promoter_terminator, sgRNA_primer_json,enzyme_df,enzyme_name,n_20_label,ccdb_label,promoter_terminator_label

In [27]:
sgRNA_primer_position_json, sgRNA_failture_primer = p_d_seq.check_locate_primer(sgRNA_plasmid_backbone, sgRNA_primer_json)
ccdb_primer_position_json, ccdb_failture_primer = p_d_seq.check_locate_primer(ccdB_plasmid_backbone, ccdb_primer_json)

In [28]:
sgRNA_promoter_terminator_start = sgRNA_plasmid_backbone.find(sgRNA_promoter_terminator)

In [ ]:
sort_compose_primer(sgRNA_promoter_terminator_start,
                    sgRNA_primer_json,
                    sgRNA_primer_position_json,
                    no_ccdb_plasmid,
                    sgRNA_plasmid_backbone,
                    n_20_label,
                    ccdb_label,
                    promoter_terminator_label)

In [54]:
sgRNA_region_seq_json = {
    'region1':"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt",
    'region2':"aaagatgctgaagatcagttgggtgcacgagtgggttacatcgaactggatctcaacagcggtaagatccttgagagttttcgccccgaagaacgttttccaatgatgagcacttttgcttcctcgctcactgact"
}


In [55]:

sgRNA_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'
ccdb_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'


def convert(gb_path,region_seq_json):
    gb = SeqIO.read(gb_path, "genbank")
    gb_seq = str(gb.seq).upper()
    
    region_cor_json = {}
    for k,v in region_seq_json.items():
        v = v.upper()
        start = gb_seq.find(v)
        end = start + len(v)
        if len(gb_seq) < end:
            end = end - len(gb_seq)
        region_cor_json.update({k:f'{start},{end}'})
        
    return region_cor_json

In [56]:
sgRNA_region_json = convert(sgRNA_gb_path,sgRNA_region_seq_json)

In [57]:
sgRNA_region_json

{'region1': '11299,11346', 'region2': '4986,5122'}

In [20]:
ccdb_region_json = convert(sgRNA_gb_path,ccdb_region_seq_json)

In [21]:
ccdb_region_json

{'region1': '-1,130', 'region2': '-1,152'}

In [15]:
ccdb_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'

gb = SeqIO.read(ccdb_gb_path, "genbank")
gb_seq = str(gb.seq)

In [ ]:
TCATCGACTT

In [75]:
su.revComp('ATTTCACAC')[::-1]

'TAAAGTGTG'

In [62]:
region_json = {'region1': 'ATTTCACAC'}

In [73]:
region_json = {'region1': 'GTGTGAAAT'}

In [114]:
region_json = {'region1': 'ATTTCACAC','region2':'dhbsjkhb'}

'CAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATA'

In [100]:
cor_json_plus

{'region1': '-1,-1'}

In [98]:
gb_path =  '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'

cor_json_plus = convert_seq_cor(gb_path,region_json,strand='+')
cor_json_min = convert_seq_cor(gb_path,region_json,strand='-')

In [115]:

aa = check_seq_in_gb(gb_path,region_json)   

In [116]:
UHA_ARGS
SEQ_ALTERED_ARGS
DHA_ARGS
UP_SGRNA_ARGS
DOWN_SGRNA_ARGS

PLASMID_Q_ARGS
GENOME_Q_ARGS

,region1,region2
0,ATTTCACAC,The sequence you provided is not on the plasmi...


In [105]:
def check_seq_in_gb(gb_path,seq_json):
    
    cor_json_plus = convert_seq_cor(gb_path,seq_json,strand='+')
    cor_json_min = convert_seq_cor(gb_path,seq_json,strand='-')
    
    for plus,minus in zip(cor_json_plus,cor_json_min):
        if cor_json_plus[plus] =='-1,-1' and  cor_json_min[minus] == '-1,-1':
            #序列不存在GB文件
             region_json[plus] = 'The sequence you provided is not on the plasmid file'
        elif cor_json_plus[plus]!='-1,-1' and cor_json_min[minus] == '-1,-1':
            #序列在正义链
            pass
        elif cor_json_plus[plus]!='-1,-1' and cor_json_min[minus] == '-1,-1':
            #序列在负义链
            region_json[plus] = su.revComp(region_json[plus])
        else:
            pass
    return seq_json

In [84]:
#只要不存在off-target，可义将一切序列转化成坐标
def convert_seq_cor(gb_path,region_seq_json,strand='+'):
    gb = SeqIO.read(gb_path, "genbank")
    gb_seq = str(gb.seq).upper()
    
    if strand == '-':
        gb_seq = su.revComp(gb_seq)
    
    region_cor_json = {}
    for k,v in region_seq_json.items():
        v = v.upper()
        start = gb_seq.find(v)
        end = start + len(v)
        if start == -1:
            i = 1
            start_seq = ''
            end_seq = ''
            while True:
                new_start = gb_seq.find(v[:i])
                if new_start == -1:
                    break
                else:
                    if gb_seq.find(v[i:]) == 0:
                        start_seq = v[:i]
                        end_seq = v[i:]
                        print(start_seq,end_seq)
                        break
                    else:        
                        i = i + 1
                        if i == len(v):
                            break
            if start_seq == '' and end_seq == '':
                start = -1
                end = -1
            else:
                start = gb_seq.find(start_seq)
                end = gb_seq.find(end_seq)
                end = end + len(v)
                
        region_cor_json.update({k:f'{start},{end}'})
        
    return region_cor_json  

In [ ]:
# 导入Biopython的SeqIO模块
from Bio import SeqIO

# 创建一个SeqRecord对象，表示DNA序列
dna_record = SeqIO.SeqRecord(dna_seq)

# 设置SeqRecord对象的属性
dna_record.id = "DNA1"
dna_record.name = "DNA1"
dna_record.description = "A sample DNA sequence with primers"

# 导入Biopython的SeqFeature模块
from Bio import SeqFeature

# 创建两个SeqFeature对象，表示引物序列在DNA序列上的位置和方向
primer1_feature = SeqFeature.SeqFeature(SeqFeature.FeatureLocation(0, len(primer1)), type="primer_bind", strand=+1)
primer2_feature = SeqFeature.SeqFeature(SeqFeature.FeatureLocation(len(dna_seq) - len(primer2), len(dna_seq)), type="primer_bind", strand=-1)

# 把SeqFeature对象添加到SeqRecord对象的features列表中
dna_record.features.append(primer1_feature)
dna_record.features.append(primer2_feature)

# 把SeqRecord对象写入一个新的gb文件中
with open("output.gbk", "w") as output_handle:
    SeqIO.write(dna_record, output_handle, "genbank")